##**Importing Libraries**


In [6]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2

###**Data Preprocessing**

In [56]:
dataFrame = pd.read_csv('fer2013.csv')
y_train=dataFrame.iloc[:20000,:1].values
X=dataFrame.iloc[:20000,1:2].values
x_train = []

for i in range(20000):
    if y_train[i][0]==1:
        y_train[i][0]=0
    if y_train[i][0]==4:
        y_train[i][0]=2
    if y_train[i][0]==5:
        y_train[i][0]=1
    if y_train[i][0]==6:
        y_train[i][0]=4
        
y_train=np_utils.to_categorical(y_train)        
        
for i in range(20000):
    pixels = np.array(X[i][0].split(" "),dtype='float32')
    pixels=np.reshape(pixels,(48,48,1))
    x_train.append(pixels)


x_train=np.array(x_train,'float32')/255.0


## **Building the Model**


In [74]:
num_features=64


model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(num_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_20 (Batc (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
batch_normalization_21 (Batc (None, 20, 20, 128)      

### *Compling and Testing*

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])
hist=model.fit(x_train,y_train,shuffle=1,epochs=30,batch_size=64,validation_split=0.2,callbacks=[chkp,es])

## *Saving the weights*

In [69]:
model.save_weights('fer_weight.h5',overwrite=1)

In [76]:
model.load_weights('best_model.h5')

## Callbacks

In [78]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es = EarlyStopping(monitor='val_acc',patience=2)
chkp = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)